In [6]:
import pymongo
from pymongo import MongoClient
import csv
import os
from petl import fromcsv, look, cut, tocsv 
import numpy as np
from StringIO import StringIO
import sklearn
from sklearn import linear_model
import plotly 
import plotly.plotly as py
import re
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from geopy.distance import vincenty
from scipy.stats import norm
from numpy import linspace
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from pylab import plot,show,hist,figure,title
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from scipy.stats import norm
import scipy.stats
from matplotlib.legend_handler import HandlerLine2D
from astropy.table import Table, Column

In [7]:
#load and merge the tables scrapted from public datatbase
def p2f(x):
    return float(x.strip('%'))/100

county2labor = pd.read_csv('CA_county_employment.csv').as_matrix()
for i in range(1,county2labor.shape[0]):
    county2labor[i,5] = p2f(county2labor[i,5])
county2LF=county2labor[:,[0,2,5]]
for i in range(1,county2LF.shape[0]):
    county2LF[i,1]=int(county2LF[i,1])
county2LF[0,2] ='unemploy_rate'

propTax = pd.read_csv('CA_propTax.csv').as_matrix()
propTax[22,7] = '0'
propTax[58,7] = '0'

for i in range(1,propTax.shape[0]):
    propTax[i,7] = p2f(propTax[i,7])
propTax[22,7] = np.median(propTax[:,7])
propTax[58,7] = np.median(propTax[:,7])

propTax[0,0]='County'
propTax = propTax[:,[0,7]]
propTax[0,1]='midTax'

zip2county = pd.read_csv('CA_zip2county.csv').as_matrix()
zip2county = zip2county[:,[0,2]]
zip2county[0,0]='zipcd'
zip2county[0,1]='County'

for i in range(1,zip2county.shape[0]):
    zip2county[i,0]=int(zip2county[i,0])


df1=pd.DataFrame(county2LF, columns=['County', 'Labor Force', 'unemploy_rate'])
df2=pd.DataFrame(propTax, columns=['County', 'midTax'])
df3=pd.DataFrame(zip2county, columns=['zipcd', 'County'])

['zipcd', 'County','Labor Force', 'unemploy_rate', 'midTax']

df_merge=df1.merge(df2,on='County')
df_merge_merge=df3.merge(df_merge,on='County')
df_zip2countyInfo = df_merge_merge.ix[1:]
#df_zip2countyInfo =['County', 'Labor Force', 'unemploy_rate','midTax','zipcd']

In [8]:
# load the previous transaction data 
%matplotlib inline
#%qtconsole
data = pd.read_csv('soldPend_home_all.csv').as_matrix()

low_memory=False
mylist=[]
u, indices =np.unique(data[:,3], return_index=True)
data_uni = data[indices,:]
data=data_uni
                            

idx=[0,1,5,7,18]
data[:,idx]=data[:,idx]/1000 #convert price unit to $k
data[:,11]=2016-data[:,11] #convert year_built to age

idx_forFlip=np.where(data[:,16]==True)
data[:,16]=1
data[idx_forFlip,16]=2

idx_takedown=np.where(data[:,6]==True)
data[:,6]=1
data[idx_takedown,6]=2


#Handeling the non-exist and mis-input zipcode. Currently just not using those data. 
#It can be modified in the future to get zipcode from latitude and longitude
# if those are available.
noZip = pd.isnull(data[:,2])
idx_noZip=np.where(noZip==True)
data = np.delete(data,idx_noZip,0)

for i in range(0,data.shape[0]):
    if (type(data[i,2]) is str):
        #print data[i,2]
        data[i,2] = int(data[i,2][:5])
        #print data[i,2]
idx_misZip=np.where(data[:,2]<90000)   
data = np.delete(data,idx_misZip,0)


idx_soldyearFilter=np.where(data[:,13]>10000)
data=np.delete(data,idx_soldyearFilter,0)
idx_lotFilter=np.where(data[:,12]>2*pow(10,11))
data=np.delete(data,idx_lotFilter,0)
idx_sqftFilter=np.where(data[:,17]>400000)
data=np.delete(data,idx_sqftFilter,0)
idx_misSqft=np.where(data[:,17]<20)
data=np.delete(data,idx_misSqft,0)
                 
idx_priceFilter=np.where(data[:,18]>200000)
data=np.delete(data,idx_priceFilter,0)

#handling the unreasonable and irrelevant year_built
idx_yearFilter=np.where(data[:,11]<0)
data[idx_yearFilter,11]=np.median(data[:,11])
idx_yearFilter=np.where(data[:,11]>500)
data[idx_yearFilter,11]=np.median(data[:,11])

data_forComp=data

col_null = pd.isnull(data[:,1])
idx_prepriceExist=np.where(col_null==True)
data=np.delete(data,idx_prepriceExist,0)



/Users/Cindy/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [9]:
#imputing #bed and #bath
numBath_mis=np.where(data[:,4]>100) #remove an outlier in the numBath
data=np.delete(data,numBath_mis,0)

noBath = pd.isnull(data[:,4])
idx_noBath=np.where(noBath==True)
data[idx_noBath,4]=np.median(data[:,4])

idx_0Bath = np.where(data[:,4]==0)
data[idx_0Bath,4]=np.median(data[:,4])

noBed = pd.isnull(data[:,10])
idx_noBed=np.where(noBed==True)
data[idx_noBed,9]=np.median(data[:,10])

idx_0Bed = np.where(data[:,10]==0)
data[idx_0Bed,9]=np.median(data[:,10])

#drop the rows where there are still features missing
for i in range(0,19):
    col_null = pd.isnull((data[:,i]))
    idx=np.where(col_null==True)
    data=np.delete(data,idx,0)

In [10]:
# merge the previous transaction data with transaction data

df=pd.DataFrame(data, columns=['pre_sqft_price','pre_price','zipcd','propID','numBath','sqft_price','takedown',\
                               'cur_lis_price','long','lat','numBed','year_built','lot_sqft','numDays','year',\
                               'month','forFlip','sqft','price'])

df_statbyZip = df['price'].astype(float).groupby(df['zipcd']).agg([np.median,np.mean,np.std,np.max,np.min,len])

data_statbyZip = df_statbyZip.as_matrix()
b=(np.unique(data[:,2])).reshape((df_statbyZip.shape[0],1))
data_statbyZip=np.append(data_statbyZip,b,1)
#df_zip2countyInfo =['County', 'Labor Force', 'unemploy_rate','midTax','zipcd']
df_statbyZip=pd.DataFrame(data_statbyZip, columns=['median','mean','std','max','min','count','zipcd'])
df_additional = df_statbyZip.merge(df_zip2countyInfo,on='zipcd')
df_all=df.merge(df_additional,on='zipcd')
data_additional = df_additional.as_matrix()
data_all=df_all.as_matrix()
data_all[:,[18,28]]=data_all[:,[28,18]]
data_all[:,[0,3]]=data_all[:,[3,0]]
data_all[:,[1,2]]=data_all[:,[2,1]]
data_all[:,[8,9]]=data_all[:,[9,8]]
#remove houses that has most likely mis input of sqft and which that has unreasonalbe price per sqft

var_noNeed=[3,5,7,20,21,22,23,25]
data_model=np.delete(data_all,var_noNeed,1)  #delete the variable that are not needed


In [3]:
# to disply the columes in data_all
fields_data_all=['propID','zipcd','pre_price','pre_sqft_price','numBath','sqft_price','takedown','cur_lis_price',\
             'long','lat','numBed','year_built','lot_sqft','numDays','year','month',\
             'forFlip','sqft','midTax','median','mean','std','max','min','len','County','Labor Force',\
        'unemploy_rate','price']
for index in np.arange(len(fields_data_all)):
    print str(index) + "," + fields_data_all[index]

0,propID
1,zipcd
2,pre_price
3,pre_sqft_price
4,numBath
5,sqft_price
6,takedown
7,cur_lis_price
8,long
9,lat
10,numBed
11,year_built
12,lot_sqft
13,numDays
14,year
15,month
16,forFlip
17,sqft
18,midTax
19,median
20,mean
21,std
22,max
23,min
24,len
25,County
26,Labor Force
27,unemploy_rate
28,price


In [4]:
# to disply the columes in data_model
fields_data_model=['propID','zipcd','pre_price','numBath','takedown',\
             'long','lat','numBed','year_built','lot_sqft','numDays','year','month',\
             'forFlip','sqft','midTax','median','len','Labor Force','unemploy_rate','price']
for index in np.arange(len(fields_data_model)):
    print str(index) + "," + fields_data_model[index]

0,propID
1,zipcd
2,pre_price
3,numBath
4,takedown
5,long
6,lat
7,numBed
8,year_built
9,lot_sqft
10,numDays
11,year
12,month
13,forFlip
14,sqft
15,midTax
16,median
17,len
18,Labor Force
19,unemploy_rate
20,price


In [48]:
#enter the info for the house of interest

zipcd_test=90012
#df_additional=['median','mean','std','max','min','count','zipcd',County','Labor Force', 'unemploy_rate','midTax']
idx_zip=np.where(data_additional[:,6]==zipcd_test)
loc_test = [0, zipcd_test,850,1,1, 34.065958,-118.246625,2,2016-1958,6976,0,2016,6,1,1196,data_additional[idx_zip,10],\
            data_additional[idx_zip,0],data_additional[idx_zip,5],data_additional[idx_zip,8], data_additional[idx_zip,9]]

#fields_data_model=['propID','zipcd','list_price','numBath','takedown',\
             #'lat','long','numBed','year_built','lot_sqft','numDays','year','month',\
'''
zipcd_test=90039
idx_zip=np.where(data_additional[:,6]==zipcd_test)
loc_test = [0, zipcd_test,999,2.5,1, 34.086281,-118.247673,4,2016-1990,3743*2,0,2016,6,1,3743,data_additional[idx_zip,10],\
            data_additional[idx_zip,0],data_additional[idx_zip,5],data_additional[idx_zip,8], data_additional[idx_zip,9]]
         #'forFlip','sqft','midTax','median','len','Labor Force','unemploy_rate','price']
'''
'''   
zipcd_test=90045
idx_zip=np.where(data_additional[:,6]==zipcd_test)
loc_test = [0, zipcd_test,1599,3,1, 33.978176,-118.402063,4,2016-1993,5616,0,2016,6,1,2478,data_additional[idx_zip,10],\
            data_additional[idx_zip,0],data_additional[idx_zip,5],data_additional[idx_zip,8], data_additional[idx_zip,9]]


'''


'   \nzipcd_test=90045\nidx_zip=np.where(data_additional[:,6]==zipcd_test)\nloc_test = [0, zipcd_test,1599,3,1, 33.978176,-118.402063,4,2016-1993,5616,0,2016,6,1,2478,data_additional[idx_zip,10],            data_additional[idx_zip,0],data_additional[idx_zip,5],data_additional[idx_zip,8], data_additional[idx_zip,9]]\n\n\n'

In [45]:
#model 1
def price_Est_model1():
    Y_train = data_model[:,20]-data_model[:,2]# for sold price, 20; for list price, 2
    loc_train = data_model[:,:20]

    dist_miles=[]
    for i in range(0,loc_train.shape[0]):
        dist_miles.append(vincenty(loc_test[5:7], loc_train[i,5:7]).miles)

    dist=np.asarray(dist_miles)
    dist_sorted = np.sort(dist)

    top = 5000
    dist_selected = dist_sorted[0:top]

    loc_match_id=[np.where(dist == dist_selected[x])[0] for x in range(0,top)]
    loc_match_id = np.concatenate(loc_match_id).ravel()
    loc_match_id = np.unique(loc_match_id)

    feature1=loc_train[loc_match_id,2:]
    dist_feature = dist[loc_match_id]
    Y_train = Y_train[loc_match_id]

    feature=np.vstack((feature1.T, dist_feature.T))
    feature=feature.T    

    idx_priceFilter = np.where(Y_train[:]>3*loc_test[2])
    idx_priceFilter = np.asarray(idx_priceFilter)
    feature=np.delete(feature,idx_priceFilter,0)
    Y_train=np.delete(Y_train,idx_priceFilter,0)

    idx_sqftFilter = np.where(feature[:,12]>3*loc_test[14])
    idx_sqftFilter = np.asarray(idx_sqftFilter)
    feature=np.delete(feature,idx_sqftFilter,0)
    Y_train=np.delete(Y_train,idx_sqftFilter,0)

    idx_lotsqftFilter = np.where(feature[:,7]>3*loc_test[9])
    idx_lotsqftFilter = np.asarray(idx_lotsqftFilter)
    feature=np.delete(feature,idx_lotsqftFilter,0)
    Y_train=np.delete(Y_train,idx_lotsqftFilter,0)

    idx_numDaysFilter = np.where(feature[:,8] < loc_test[10])
    idx_numDaysFilter = np.asarray(idx_numDaysFilter)
    feature=np.delete(feature,idx_numDaysFilter,0)
    Y_train=np.delete(Y_train,idx_numDaysFilter,0)     

    feature_test =np.append(loc_test[2:], 0)

    if feature.shape[0]==0:
        feature = feature_test.reshape((1,feature_test.shape[0]))
        Y_train = np.asarray([1])    
    feature2 = np.vstack(feature[:, 1:]).astype(np.float) #to not using list price as one of the feature
    feature_test2 = np.vstack(feature_test[1:]).astype(np.float) #to not using list price as one of the feature

    #feature2 = np.vstack(feature[:,:]).astype(np.float) #to use list price as one of the feature
    #feature_test2 = np.vstack(feature_test).astype(np.float) #to use list price as one of the feature

    scaler = preprocessing.StandardScaler().fit(feature2)
    X_scaled = scaler.transform(feature2)  
    X_scaled_normed = preprocessing.normalize(X_scaled, norm='l2')

    X_test_scaled = scaler.transform(feature_test2.reshape(1, -1))  
    X_test_scaled_normed= preprocessing.normalize(X_test_scaled.reshape(1, -1) , norm='l2')
    X_test_scaled_normed = X_test_scaled_normed[:,:]

    from sklearn.preprocessing import PolynomialFeatures
    poly = PolynomialFeatures(1,interaction_only=True)
    X_train = poly.fit_transform(X_scaled_normed)  
    X_test = poly.fit_transform(X_test_scaled_normed.reshape(1, -1) )  

    for i in range(1,X_train.shape[1]):
        col_null = pd.isnull((X_train[:,i]))
        idx=np.where(col_null==True)
        X_train=np.delete(X_train,idx,0)
        Y_train=np.delete(Y_train,idx,0)
        col_null2 = pd.isnull((Y_train[:]))
        idx2=np.where(col_null2==True)
        X_train=np.delete(X_train,idx2,0)
        Y_train=np.delete(Y_train,idx2,0)

    #print ('the training set size is: [') + str(X_train.shape[0]) +',' + str(X_train.shape[1]) +']'
    rf=RandomForestRegressor()
    #RR=linear_model.Ridge(alpha =1) 
    #svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
    clf= rf
    clf.fit(X_train,Y_train)
    diff_est=clf.predict(X_test)
    price_est=diff_est+loc_test[2]
    return price_est

price_est_model1=[]
for i in range(0,10,1):
    price_est=price_Est_model1()
    price_est_model1.append(price_est)

print ('the estimated price from model 1 is: ') + str(np.mean(price_est_model1))

the estimated price from model 1 is: 1592.95038


In [49]:
# model 2

Y_train_compa = data_model[:,20]
feature1_compa=data_model[:,:20]

idx_numDaysFilter_compa = np.where(feature1_compa[:,10]>(loc_test[10]+180))
idx_numDaysFilter_compa = np.asarray(idx_numDaysFilter_compa)
feature1_compa=np.delete(feature1_compa,idx_numDaysFilter_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_numDaysFilter_compa,0)

idx_numDaysFilter2_compa = np.where(feature1_compa[:,10] < loc_test[10])
idx_numDaysFilter2_compa = np.asarray(idx_numDaysFilter2_compa)
feature1_compa=np.delete(feature1_compa,idx_numDaysFilter2_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_numDaysFilter2_compa,0)   

idx_priceFilter_compa = np.where(np.absolute(np.divide(Y_train_compa-loc_test[2],loc_test[2]))>0.1)
idx_priceFilter_compa = np.asarray(idx_priceFilter_compa)
feature1_compa=np.delete(feature1_compa,idx_priceFilter_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_priceFilter_compa,0)

idx_sqftFilter_compa = np.where(np.abs(np.divide(feature1_compa[:,14]-loc_test[14],loc_test[14]))>0.2)
idx_sqftFilter_compa = np.asarray(idx_sqftFilter_compa)
feature1_compa=np.delete(feature1_compa,idx_sqftFilter_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_sqftFilter_compa,0)

idx_flipFilter_compa = np.where(feature1_compa[:,13] != loc_test[13])
idx_flipFilter_compa = np.asarray(idx_flipFilter_compa)
feature1_compa=np.delete(feature1_compa,idx_flipFilter_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_flipFilter_compa,0)

idx_tdFilter_compa = np.where(feature1_compa[:,4] != loc_test[4])
idx_tdFilter_compa = np.asarray(idx_tdFilter_compa)
feature1_compa=np.delete(feature1_compa,idx_tdFilter_compa,0)
Y_train_compa=np.delete(Y_train_compa,idx_tdFilter_compa,0)

dist_compa=[]
for i in range(0,feature1_compa.shape[0]):
    dist_compa.append(vincenty(loc_test[5:7], feature1_compa[i,5:7]).miles)
    
dist_compa=np.asarray(dist_compa)
dist_compa_sorted = np.sort(dist_compa)

top_compa= 30
dist_select_compa= dist_compa_sorted[0:top_compa]
loc_match_id_compa=[np.where(dist_compa == dist_select_compa[x])[0] for x in range(0,top_compa)]
loc_match_id_compa = np.concatenate(loc_match_id_compa).ravel()
loc_match_id_compa = np.unique(loc_match_id_compa)
dist_selected_compa = dist_compa_sorted[0:top_compa]
dist_feature_compa = dist_compa[loc_match_id_compa]
Y_train_compa = Y_train_compa[loc_match_id_compa]
feature_test_compa =np.append(loc_test, 0)

feature1_compa = feature1_compa[loc_match_id_compa,:]
feature_compa=np.vstack((feature1_compa.T, dist_feature_compa.T))
feature_compa=feature_compa.T

feature_compa = np.vstack(feature_compa[:, 3:]).astype(np.float)
feature_test_compa = np.vstack(feature_test_compa[3:]).astype(np.float)

scaler_compa = preprocessing.StandardScaler().fit(feature_compa)
X_scaled_compa = scaler_compa.transform(feature_compa)  
X_scaled_normed_compa = preprocessing.normalize(X_scaled_compa, norm='l2')

X_train_compa = X_scaled_normed_compa

X_test_scaled_compa = scaler_compa.transform(feature_test_compa.reshape(1, -1))  
X_test_scaled_normed_compa= preprocessing.normalize(X_test_scaled_compa.reshape(1, -1) , norm='l2')
X_test_compa = X_test_scaled_normed_compa

print str(X_train_compa.shape[0]) + (' houses are used for comparable house search')

import math
from itertools import izip

def dot_product(v1, v2):
    return sum(map(lambda x: x[0] * x[1], izip(v1, v2)))

def cosine_measure(v1, v2):
    prod = dot_product(v1, v2)
    len1 = math.sqrt(dot_product(v1, v1))
    len2 = math.sqrt(dot_product(v2, v2))
    return prod / (len1 * len2)

similarity = []
for i in range(0,X_train_compa.shape[0]):
    similarity.append(cosine_measure(X_test_compa.reshape((-1,1)), X_train_compa[i,:].reshape((-1,1))) )

    
    
# find the top 10 similar houses based on cosine similarity 
from astropy.table import Table, Column
top_simi = 10

simi_sorted = sorted(similarity, reverse=True)
simi_select = simi_sorted[0:top_simi]
simi_match_id =[np.where(similarity == simi_select[x])[0] for x in range(0,top_simi)]
simi_match_id = np.concatenate(simi_match_id).ravel()
simi_match_id = np.unique(simi_match_id)

numBath = feature_compa[simi_match_id, 0].astype(np.uint8)
numBed = feature_compa[simi_match_id, 4].astype(np.uint8) 
year_built = 2016-feature_compa[simi_match_id, 5].astype(np.uint16) 
lot_sqft = feature_compa[simi_match_id, 6] 
sold_year = feature_compa[simi_match_id, 8].astype(np.uint16) 
sold_month =feature_compa[simi_match_id, 9].astype(np.uint8) 
sqft=feature_compa[simi_match_id, 11] 
prop_tax_rate = feature_compa[simi_match_id, 12] 
count_perzip = feature_compa[simi_match_id, 14].astype(np.uint16) 
distance=feature_compa[simi_match_id, 17]
zipcd = feature1_compa[simi_match_id, 1]
propID = feature1_compa[simi_match_id, 0]


simi = np.asarray(similarity)[simi_match_id]

for i in range(0,simi_match_id.shape[0]):
    simi[i]=float(int(simi[i]*100))/100
    distance[i]=float(int(distance[i]*10))/10
    
simi=simi.reshape((-1,1))    
sold_price = Y_train_compa[simi_match_id]

t = Table([simi,propID, zipcd, numBath, numBed,year_built,lot_sqft,sold_year,sold_month,sqft,\
           prop_tax_rate,distance,sold_price], names=('similarity','propID','zipcd','#Bath', '#Bed',\
                                                            'yr._built','lot','sold_yr.',\
                                                            'sold_mon.','sqft','prop_taxrate','dist',\
                                                            'sold_price'))
weight = np.asarray(simi/np.sum(simi)).reshape((10,))
price_est_weighted=np.sum(np.multiply(sold_price,weight))
price_est_median=np.median(sold_price)
print ('the estimated price from model 2 is: ') + str(price_est_weighted)

t_propID = Table([propID,year_built], names=('propID','year_built'))

import csv

with open('top10comp_#825.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    [writer.writerow(r) for r in t_propID]

t

30 houses are used for comparable house search
the estimated price from model 2 is: 838.924479167


similarity [1],propID,zipcd,#Bath,#Bed,yr._built,lot,sold_yr.,sold_mon.,sqft,prop_taxrate,dist,sold_price
float64,object,object,uint8,uint8,uint16,float64,uint16,uint8,float64,float64,float64,object
0.41,5666513e6b16b194960000d9,90026,2,2,1913,5986.0,2016,5,1092.0,0.0129,1.3,895.0
0.51,56cfa3ce6b16b1380ee7ce2f,90026,2,3,1926,7826.0,2016,6,1134.0,0.0129,0.7,795.0
0.38,56fd68c7ec0c19f4218fb4e0,90026,1,2,1952,3431.0,2016,5,1320.0,0.0129,2.1,920.0
0.31,57025f28ec0c191c7ba37f3a,90026,1,2,1925,3000.0,2016,5,1110.0,0.0129,2.1,785.0
0.27,5704f837ec0c19ef078826a2,90026,2,3,1915,4360.0,2016,5,1300.0,0.0129,1.6,799.0
0.55,570d8632ec0c19f69f22bcd0,90026,1,2,1953,11040.0,2016,6,1164.0,0.0129,1.5,862.0
0.29,55b17dc169702d13bd000041,90039,2,2,1950,4956.0,2016,5,1316.0,0.0129,2.6,895.0
0.27,5637e9f66b16b11653000233,90039,2,3,1925,4999.0,2016,6,1250.0,0.0129,2.4,785.0
0.34,56df14006b16b155d785edd5,90039,1,2,1937,4728.0,2016,5,1044.0,0.0129,2.6,845.0
